In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install virtualenv

In [ ]:
!virtualenv /content/drive/MyDrive/colab_env

In [ ]:
"""
activating colab_env virtual folder from my google drive
for installing necessary libraries to avoid reinstallig them.
"""

!source /content/drive/MyDrive/colab_env/bin/activate; pip install bitsandbytes accelerate transformers torch gradio

In [ ]:
# lets find the installed packages from our colab env inside the drive
# copy your own path into it from your drive/mydrive/colab_env

import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.11/site-packages")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gradio as gr

# HF Spaces Optimized Configuration
model_name = "Qwen/CodeQwen1.5-7B-Chat"

# 1. Load tokenizer (unchanged)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# 2. Load model with HF Spaces optimizations (key changes)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                  # Critical for CPU deployment
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,      # Added for 4-bit
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval()

In [ ]:
# this is suitable for quantized version.

def generate_unit_test(code: str, framework: str = "pytest") -> str:
    """
    Generates unit tests using Qwen's official format, optimized for 4-bit quantization.
    Returns ONLY the test code without explanations.
    """
    messages = [
        {
            "role": "user",
            "content": f"""Generate a complete {framework} unit test for this Python code.
Follow these rules STRICTLY:
1. Include all necessary imports
2. Only return the executable test code
3. No explanations or markdown formatting
4. Cover edge cases

Code:
```python
{code}
```"""
        }
    ]

    # Apply chat template with quantization-safe settings
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    )

    # Quantization-compatible input processing
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}

    # Generation parameters optimized for 4-bit
    outputs = model.generate(
        **inputs,
        max_new_tokens=384,  # Slightly reduced for CPU/quantized performance
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        temperature=0.3,  # Slightly higher than original for quantized model
        top_k=40,  # Helps maintain quality with quantization
        repetition_penalty=1.15  # Prevents duplicate assertions
    )

    # Extract and clean output
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    test_code = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # Robust output cleaning (preserves imports and assertions)
    test_code = test_code.replace("```python", "").replace("```", "").strip()
    if test_code.startswith("import") or test_code.startswith("from"):
        return test_code
    return f"import {framework}\n\n" + test_code  # Fallback import

In [ ]:
# Gradio Interface

with gr.Blocks(title="🧪 Function to Unit Tests") as demo:
    gr.Markdown("""
    # 🚀 Python Function → Unit Test Generator
    Paste your Python function below and get instant unit tests!
    """)

    with gr.Row():
        with gr.Column():
            function_input = gr.Code(
                label="Your Python Function",
                language="python",
                lines=10,
                value="def add(a, b):\n    return a + b"  # Default example
            )
            framework = gr.Dropdown(
                ["pytest", "unittest"],
                label="Test Framework",
                value="pytest"
            )
            btn = gr.Button("Generate Tests", variant="primary")

        with gr.Column():
            test_output = gr.Code(
                label="Generated Unit Tests",
                language="python",
                lines=15,
                interactive=True
            )

    # Examples
    examples = [
        ["def multiply(a, b):\n    return a * b", "pytest"],
        ["def is_even(n):\n    return n % 2 == 0", "unittest"]
    ]
    gr.Examples(examples, inputs=[function_input, framework])

    btn.click(
        fn=generate_unit_test,  # ← Your original function
        inputs=[function_input, framework],
        outputs=test_output,
        api_name="generate"
    )

# Launch with sharing
demo.launch(share=True)

In [ ]:
# Congrats!